In [1]:
import tensorflow as tf
import os
import pickle
import numpy as np

CIFAR_DIR = "./cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

['data_batch_1', 'readme.html', 'batches.meta', 'data_batch_2', 'data_batch_5', 'test_batch', 'data_batch_4', 'data_batch_3']


In [2]:
def load_data(filename):
    """read data from data file."""
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']

# tensorflow.Dataset.
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 - 1
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        # [0,1,2,3,4,5] -> [5,3,2,4,0,1]
        #将0-self._num_example的数进行一个混合排列----数据及下标的混合排列
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels

train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [3]:
#残差链接块抽象出来形成一个函数
def residual_block(x,output_channel):
    '''residual connection implementation '''
    input_channel = x.get_shape().as_list()[-1]
    if input_channel*2 == output_channel:
        increase_dim = True #要不要增加通道的数目
        strides = (2,2)
    elif input_channel == output_channel:
        increase_dim = False
        strides=(1,1)
    else:
        raise Exception("input channel can't match output channel")
    conv1 = tf.layers.conv2d(x,
                                       output_channel,
                                       (3,3),
                                        strides = strides,
                                        padding = 'same',
                                        activation = tf.nn.relu,
                                        name = 'conv1')
    conv2 = tf.layers.conv2d(conv1,
                                        output_channel,
                                        (3,3),
                                        strides=(1,1),
                                        padding='same',
                                        name = 'conv2')
    if increase_dim:
        #[None , img_width , img_hight, img_channel]------>[,,,img_channel*2]
        pooled_x = tf.layers.average_pooling2d(x,
                                                              (2,2),
                                                              strides=(2,2),
                                                              padding='valid')
        
        #补零该怎么补，维度上左右补，这里有四个维度，里面分别左补一部分，又补一部分
        padded_x = tf.pad(pooled_x,
                                 [[0,0],
                                  [0,0],
                                  [0,0],
                                  [input_channel//2, input_channel//2]])
    else:
        padded_x = x
    output_x = conv2 + padded_x
    return output_x
'''残差链接块定义完成可用来搭建残差网络'''
def res_net(x, num_residual_block, num_filter_base, class_num):
#     - x:
#             - num_residual_block :一个列表代表每一层上block的多少【3，4，6，3】：共四层参考34层的残差网络
#             - num_subsampling, num_filter_base：
#             - class_num
    num_subsampling = len(num_residual_block )
    layers = []
    #[None,width,higth,channel]------>[width,higth,channel]
    input_size = x.get_shape().as_list()[1:]
    '''tf.name_scope()主要用于管理图中各种op，而tf.variable_scope()主要用于管理图中变量的名字，在 tf.name_scope下时，tf.get_variable()创建的变量名不受 name_scope 的影响（不受它的约束）'''
    with tf.variable_scope('conv0'):
        conv0 = tf.layers.conv2d(x,
                                 num_filter_base,
                                (3,3),
                                strides=(1,1),
                                padding='same',
                                activation= tf.nn.relu,
                                name='conv0')
        layers.append(conv0)#用在这里
    for sampld_id in range(num_subsampling):#每个stage
        for i in range(num_residual_block[sampld_id]):
            with tf.variable_scope('conv%d_%d'%(sampld_id,i)):
                conv = residual_block(layers[-1],
                                     num_filter_base * (2 ** sampld_id))#用来实现每个stage的输出的通道数,
                layers.append(conv)
    #做一下预测
    muliplier = 2**(num_subsampling - 1)
#     assert layers[-1].get_shape().as_list()[1:] == [input_size[0]/muliplier,input_size[1]/muliplier,num_filter_base*muliplier]
    
    #所有残差结束之后
    with tf.variable_scope('fc'):
        #[None, img_width,img_hight, img_channel]
        global_pool = tf.reduce_mean(layers[-1],[1,2])
        logits = tf.layers.dense(global_pool,class_num)
        layers.append(logits)
    return layers[-1]

        
        
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])
# [None], eg: [0,5,6,3]
x_image = tf.reshape(x, [-1, 3, 32, 32])





y_ = res_net(x_image,[2,3,2],32,10)

loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# y_ -> sofmax
# y -> one_hot
# loss = ylogy_

# indices
predict = tf.argmax(y_, 1)
# [1,0,1,1,1,0,0,0]
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [4]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 100
test_steps = 100

# train 10k: 71.35%
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op],
            feed_dict={
                x: batch_data,
                y: batch_labels})
        if (i+1) % 100 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' 
                  % (i+1, loss_val, acc_val))
        if (i+1) % 1000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data, 
                        y: test_batch_labels
                    })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test ] Step: %d, acc: %4.5f'
                  % (i+1, test_acc))

[Train] Step: 100, loss: 2.08958, acc: 0.20000
